## Import Libraries

In [ ]:
%matplotlib notebook

import pandas as pd
import gensim
import gensim.models
from gensim.models.callbacks import CallbackAny2Vec
import numpy as np
from matplotlib import pyplot as plt

## Input text-based data
I process my data into csv format with each document labeled by title, year, and then content

In [ ]:
df = pd.read_csv('/Users/codyotoole/Desktop/Research_Data/GeneDrive_Data/gd_allyears_clean.csv')

## A bit of processing
In my data a word of interest has multiple different uses, so I standardize them so the neural net interprets them as the same word

In [ ]:
for i in range(0,len(df)):
    df['word'][i] = df['word'][i].replace('gene drive', 'gene_drive')

for i in range(0,len(df)):
    df['word'][i] = df['word'][i].replace('gene-drive', 'gene_drive')

for i in range(0,len(df)):
    df['word'][i] = df['word'][i].replace('Gene drive', 'gene_drive')

for i in range(0,len(df)):
    df['word'][i] = df['word'][i].replace('genetic drive', 'gene_drive')

## Split Dataset into Time Segments
I am splitting the data into multiple data frames where each dataframe corresponds to a single year

In [ ]:
dfs = {}
for i, n in zip(range(1,17), range(2003, 2019)):
    dfs[i] = df.loc[df['Date'] == n]
    dfs[i].reset_index(drop=True, inplace=True)

## Word2Vec Models

In [ ]:
#log epochs 
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    
    def __init__(self):
         self.epoch = 0
         
    def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
         print("Epoch #{} end".format(self.epoch))
         self.epoch += 1

### This makes a single w2v model for each time period

In [ ]:
m = {}
for i in range(1,17):
    words = []
    for n in range(0, len(dfs[i])):
        w = dfs[i]["word"][n].split()
        words.append(w)
    m[i] = gensim.models.Word2Vec(min_count=1, window=5, size=100, 
                                   sample=1e-4, negative=10, workers=10)
    m[i].build_vocab(words)  
    epoch_logger = EpochLogger()
    m[i].train(words, epochs =50, total_examples=m[i].corpus_count, callbacks=[epoch_logger])

## We need to align the gensim models to compare across time
code pulled from https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf

In [ ]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        (With help from William. Thank you!)

    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.

    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """
    
    # patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
    base_embed.init_sims()
    other_embed.init_sims()

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

    # get the embedding matrices
    base_vecs = in_base_embed.wv.syn0norm
    other_vecs = in_other_embed.wv.syn0norm

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one
    # i.e. multiplying the embedding matrix (syn0norm)by "ortho"
    other_embed.wv.syn0norm = other_embed.wv.syn0 = (other_embed.wv.syn0norm).dot(ortho)
    return other_embed

In [ ]:
def intersection_align_gensim(m1,m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.vocab.keys())
    vocab_m2 = set(m2.wv.vocab.keys())

    # Find the common vocabulary
    common_vocab = vocab_m1&vocab_m2
    if words: common_vocab&=set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1-common_vocab and not vocab_m2-common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.vocab[w].count + m2.wv.vocab[w].count,reverse=True)

    # Then for each model...
    for m in [m1,m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.vocab[w].index for w in common_vocab]
        old_arr = m.wv.syn0norm
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.syn0norm = m.wv.syn0 = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        m.wv.index2word = common_vocab
        old_vocab = m.wv.vocab
        new_vocab = {}
        for new_index,word in enumerate(common_vocab):
            old_vocab_obj=old_vocab[word]
            new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
        m.wv.vocab = new_vocab

    return (m1,m2)

## Initialize the models

In [ ]:
#Loop initialize the models to compare across time
#keeps vocab so need to do reverse order to have largest vocab
for i in reversed(range(2,17)):
    print(i)
    m[i-1] = smart_procrustes_align_gensim(m[i], m[i-1])

## Visualization 

In [ ]:
#Cosine Similarity 
def cos_sim(a,b):
    result = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return result

#Cosine Distance
def cos_dis(a,b):
    result = 1 - cos_sim(a,b)
    return result

In [ ]:
#Set up x and y values for plot
ycos = [cos_dis(m[1]['natural'],m[i+1]['natural']) for i in range(1,16)]
ycos.insert(0,np.nan)
x=[]
for n in range(2003, 2019):
    x.append(str(n))

plot = pd.DataFrame({'x':x, 'y':ycos})

In [ ]:
#Plot Cosine Distance over time
legend_properties = {'weight':'bold'}
plt.rcParams["font.family"] = "Times New Roman"
plt.plot (plot.x, plot.y, lw=2, linestyle='--', marker='o', 
          markersize=4, color='b', label='reverse')
plt.xticks(range(0,len(x)), x)
plt.legend(prop=legend_properties)
plt.ylim([0,1])
plt.ylabel('Cosine Distance', fontsize=12)
plt.xlabel('Year', fontsize=12)
plt.title('Semantic Distance Since First Year', fontsize=12)
plt.tight_layout()